# Overview 

In the 10x series of notebooks, we will look at Time Series modeling in pycaret using univariate data and no exogenous variables. We will use the famous airline dataset for illustration. Our plan of action is as follows:

1. Perform EDA on the dataset to extract valuable insight about the process generating the time series. **(Covered in this notebook)**
2. Model the dataset based on exploratory analysis (univariable model without exogenous variables).
3. Use an automated approach (AutoML) to improve the performance.
4. User customizations, potential pitfalls and how to overcome them. 

In [1]:
!pip install pycaret

     |████████████████████████████████| 320 kB 5.0 MB/s 
     |████████████████████████████████| 16.8 MB 50.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.3 MB/s 
     |████████████████████████████████| 88 kB 8.2 MB/s 
     |████████████████████████████████| 6.8 MB 39.8 MB/s 
     |████████████████████████████████| 261 kB 47.6 MB/s 
     |████████████████████████████████| 1.7 MB 57.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 54.5 MB/s 
     |████████████████████████████████| 2.0 MB 30.0 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 116 kB 54.3 MB/s 
     |████████████████████████████████| 102 kB 11.2 MB/s 
     |████████████████████████████████| 10.9 MB 48.3 MB/s 
     |████████████████████████████████| 3.1 MB 49.5 MB/s 
     |████████████████

In [3]:
import time
import numpy as np
import pandas as pd

from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment

ModuleNotFoundError: ignored

In [ ]:
y = get_data('airline', verbose=False)

In [ ]:
# We want to forecast the next 12 months of data and we will use 3 fold cross-validation to test the models.
fh = 12 # or alternately fh = np.arange(1,13)
fold = 3

In [ ]:
# Global Plot Settings
fig_kwargs={'renderer': 'notebook'}

# Exploratory Analysis

`pycaret` Time Series Forecasting module provides a conventient interface for perform exploratory analysis using `plot_model`.

**NOTE:**
* Without an estimator argument, `plot_model` will plot using the original dataset. We will cover this in the current notebook.
* If an estimator (model) is passed to `plot_model`, the the plots are made using the model data (e.g. future forecasts, or analysis on insample residuals). We will cover this in a subsequent notebook.

Let's see how this works next. 

**First, we will plots the original dataset.**

In [ ]:
eda = TSForecastingExperiment()
eda.setup(data=y, fh=fh, fig_kwargs=fig_kwargs)

In [ ]:
# NOTE: This is the same as `eda.plot_model(plot="ts")`
eda.plot_model()  

**Let's explore the standard ACF and PACF plots for the dataset next**

In [ ]:
# ACF and PACF for the original dataset
eda.plot_model(plot="acf")

# NOTE: you can customize the plots with kwargs - e.g. number of lags, figure size (width, height), etc
# data_kwargs such as `nlags` are passed to the underlying function that gets the ACF values
# figure kwargs such as `fig_size` & `template` are passed to plotly and can have any value that plotly accepts
eda.plot_model(
    plot="pacf",
    data_kwargs={'nlags':36},
    fig_kwargs={'fig_size': [800, 500], 'template': 'simple_white'}
)

**Users may also wish to explore the spectrogram or the FFT which are very useful for studying the frequency components in the time series.**

For example:
- Peaking at f =~ 0 can indicating wandering behavior characteristic of a random walk that needs to be differenced. This could also be indicative of a stationary ARMA process with a high positive phi value.
- Peaking at a frequency and its multiples is indicative of seasonality. The lowest frequency in this case is called the fundamental frequency and the inverse of this frequency is the seasonal period for the model.

In [ ]:
eda.plot_model(plot="periodogram")
eda.plot_model(plot="fft")

**In the plots above, we notice**

1. Peaking at f ~= 0 indicating that we need to difference the data.
2. Peaking at f = 0.0833, 0.1677, 0.25, 0.3333, 0.4167. All these are multiple of 0.0833. Hence 0.0833 is the fundamental frequency and the seasonal period is 1/0.0833 = 12.

**Alternately, the `diagnostics` plot provides all these details in a convenient call.**

In [ ]:
eda.plot_model(plot="diagnostics", fig_kwargs={"height": 800, "width": 1000})

Our diagnosutic plots indicated the need to difference and the presence of a seasonal period of 12. **Let's see what happends when we remove this from the model. What other characteristics are left in the model that would need to be taken care of?** 

This can be achieved through the difference plots. Along with the difference plots, we will plot the corresponding ACF, PACF and Periodogram for further diagnostics.

In [ ]:
# Row 1: Original
# Row 2: d = 1
# Row 3: First (d = 1) then (D = 1, s = 12)
#   - Corresponds to applying a standard first difference to handle trend, and
#     followed by a seasonal difference (at lag 12) to attempt to account for
#     seasonal dependence.
# Ref: https://www.sktime.org/en/stable/api_reference/auto_generated/sktime.transformations.series.difference.Differencer.html
eda.plot_model(
    plot="diff",
    data_kwargs={
        "lags_list": [[1], [1, 12]],
        "acf": True, "pacf": True, "periodogram": True
    },
    fig_kwargs={"height": 600, "width": 1000}
)


# ## NOTE: Another way to specifi differences is using order_list
# # Row 1: Original
# # Row 2: d = 1
# # Row 3: d = 2
# eda.plot_model(
#     plot="diff",
#     data_kwargs={
#         "order_list": [1, 2],
#         "acf": True, "pacf": True, "periodogram": True
#     },
#     fig_kwargs={"height": 600, "width": 1000}
# )

**Observations:**

1. In the second row, we have only removed the wandering behavior by taking a first difference. This can be seen in the ACF plot (extended autocorrelations are gone) and Periodogram (peaking at f =~ 0 is squished). The ACF (preaking at seasonal period of 12 and its multiples) and PACF (peaking at fundamental frequency of 0.0833 and its multiples) still show the seasonal behavior.
2. In the third row, we have taken first difference followed by a seasonal difference of 12. Now, we can see that the peaking at seasonal multiples is gone from both ACF and Periodogram. There is still a little bit of autoregresssive properties that we have not taken care of but by looking at the PACF, maybe p=1 seems like a reasonable value to use (most lags after that are insignificant).

**Conclusion**
* If you were modeling this with ARIMA, the model to try would be **ARIMA(1,1,0)x(0,1,0,12)**.
* Other models could use this information appropriately. For example, reduced regression models could remove the trend and seasonality of 12 (i.e. make the data stationary) before modeling the rest of the autoregressive properties. Luckily, the `pycaret` time series module will take care of this internally.


**Let's plot the Time Series Decomposition next (another classical diagnostic plot)**

In [ ]:
# First, classical decomposition
# By default the seasonal period is the one detected during setup - 12 in this case.
eda.plot_model(plot="decomp")

# Users can chnage the seasonal period to explore what is best for this model.
eda.plot_model(plot="decomp", data_kwargs={'seasonal_period': 24})

# Users may wish to customize the decomposition, for example, in this case multiplicative seasonality
# probably makes more sense since the magnitide of the seasonality increase as the time progresses
eda.plot_model(plot="decomp", data_kwargs={'type': 'multiplicative'})

# Users can also plot STL decomposition
# Reference: https://otexts.com/fpp2/stl.html
eda.plot_model(plot="decomp_stl")

**Let us look at the various splits of the data used for modeling next.**

**NOTE:**
* In time series, we can not split the data randomly since there is serial correlation in the data and using future data to predict past data will result in leakage. Hence the temporal dependence must be maintained when splitting the data. 
* Users may wish to refer to this for more details: 
  - https://github.com/pycaret/pycaret/discussions/1761
  - https://robjhyndman.com/hyndsight/tscv/
  - https://topepo.github.io/caret/data-splitting.html#data-splitting-for-time-series

In [ ]:
# Show the train-test splits on the dataset
# Internally split - len(fh) as test set, remaining used as test set
eda.plot_model(plot="train_test_split", fig_kwargs={"height": 400, "width": 900})

# Show the Cross Validation splits inside the train set
# The blue dots represent the training data for each fold.
# The orange dots represent the validation data for each fold
eda.plot_model(plot="cv", fig_kwargs={"height": 400, "width": 900})

# Statistical Tests

Statistical Testing is another important part of time series modeling. This can be achieved easily in pycaret using `check_stats`.

**Options are:**
* 'summary',
* 'white_noise'
* 'stationarity'
* 'adf'
* 'kpss'
* 'normality'
* 'all' 

In [ ]:
# Summary Statistics
eda.check_stats(test="summary")

In [ ]:
# Stationarity tests (ADF and KPSS)
# NOTE: Users can also just run a single test by passing either 'adf' or 'kpss' to `check_stats`
eda.check_stats(test='stationarity')

The ADF tests shows that the data is not stationary and we saw this in the plots as well (clear trend and seasonal behavior)

In [ ]:
# Ljung-Bx test to tests of white noise (whether the data is uncorrelated or not)
eda.check_stats(test='white_noise')

The Ljung-Box tests indicates that the data is not white noise - again something that was clearly visible in the data

In [ ]:
# Users have the option to customize the tests such as change the alpha value.
eda.check_stats(test='kpss', alpha = 0.2)

Alternately, all the above tests can be done in one shot by not passing any test type.

In [ ]:
eda.check_stats()

**That's it for this notebook. In the next notebook, we will see how we can start to model this data.**